# ЗАДАЧИ

## ЛЕГЕНДА

Итак, вы работаете аналитиком в онлайн-школе MasterMind.

Уже в конце рабочего дня вам пишет расстроенный продакт-менеджер. Несчастный Григорий крайне устал от того, что новые курсы, созданные с той же любовью, что и прежние, не пользуются особой популярностью среди пользователей — несмотря на все усилия отдела маркетинга. 

## ЦЕЛЬ
Подготовить основу рекомендательной системы.

## ЗАДАЧИ
Итак, продакт ожидает получить от вас рекомендательную систему, благодаря которой можно будет предлагать клиентам интересные им курсы и тем самым повышать средний чек.

Вы решаете, что изначальным воплощением этой системы может стать таблица, в которой курсам будет соответствовать по две рекомендации.

Кроме того, вы планируете вместе с отчётом (таблицей рекомендаций) скинуть продакту ещё и все написанные в процессе скрипты, чтобы было меньше вопросов по решению :) Ну, и раз в код будут смотреть не только ваши глаза, вы считаете необходимым снабдить его комментариями, которые бы разъясняли, что где и почему вы делаете.

Также вы понимаете, что перед внедрением фичи коллеги решат провести A/B-тест и вас скорее всего привлекут к анализу результатов.

Перспективы ясны, можно переходить к формализации задач.

## КОНКРЕТНЫЕ ШАГИ (ФОРМАЛИЗОВАННЫЕ ЗАДАЧИ)

Обдумав план предстоящей работы, вы понимаете, что действовать нужно по привычной схеме:

1. Познакомиться с датасетом, подготовить и проанализировать данные с помощью SQL.
2. Обработать данные средствами Python.
3. Составить итоговую таблицу с рекомендациями, снабдив её необходимыми комментариями, и представить отчёт продакт-менеджеру.
4. Проанализировать результаты A/B-теста, проведённого после внедрения фичи, и сделать вывод.



Итак, у продакт-менеджера есть идея создать простую рекомендательную систему, которая будет подсказывать пользователям, какие курсы могут быть им интересны. Такая система позволит организовать допродажу в корзине и тем самым увеличить средний чек.

В ваших аналитических силах составить для маркетингового отдела такую таблицу, где с каждым курсом будут сопредставлены ещё два — которые будут рекомендоваться. 

**Таблица carts** — данные о пользовательских корзинах
- Promo Code ID — ID промокода, если он есть
- Purchased At — дата оплаты
- User ID — ID пользователя
- Created At — дата создания корзины
- Updated At — дата последнего обновления информации
- ID — идентификатор корзины
- State — состояние оплаты

**Таблица cart items** — данные о курсах, которые пользователи добавили в корзину
- Created At — дата создания события
- Resource Type — тип продукта
- Resource ID — ID курса
- Cart ID — идентификатор корзины
- Updated At — дата последнего обновления информации
- ID — идентификатор операции

In [214]:
import pandas as pd
import psycopg2

*Я выгрузил файлы для с учебной платформы и загрузил на GitHub, что бы работать с ними независимо от площадки, особено для того, что бы сделать рабочие SQL-запросы и Phyton код в одном интерфейсе файла ноутбук.*

Продажи за какие годы есть в ваших данных?

`Select
extract(YEAR from purchased_at) as y_purchase
from final.carts 
Group by 1`

ОТВЕТ 2017, 2018

Для построения рекомендательной системы полезно знать, какие курсы чаще всего покупают вместе с конкретным курсом (необязательно в рамках одной корзины).
Так вы сможете ранжировать все курсы, которые предлагает компания, и выбрать только те, что с наибольшей вероятностью заинтересуют клиентов.

Сколько клиентов покупали курсы?

`Select
Count (distinct c.User_ID) as users_num_course
from final.cart_items as ci join final.carts as c on c.ID = ci.Cart_ID
WHERE ci.Resource_Type = 'Course' and c.Purchased_At is not Null`


ОТВЕТ 49006

Сколько всего есть различных курсов?

`Select
count(distinct resource_id)
from final.cart_items
where resource_type = 'Course'`

ОТВЕТ 127

Каково среднее число купленных курсов на одного клиента?

`Select
	round(round(count(ci.Resource_ID),2) / round (count (distinct c.User_ID),2), 2)	
from final.cart_items as ci join final.carts as c on c.ID = ci.Cart_ID
	WHERE ci.Resource_Type = 'Course' 
		and c.Purchased_At is not Null`

ОТВЕТ 1.44

Сколько клиентов купили больше одного курса?

`With loyal_users as(
	Select
		c.User_ID
	from final.cart_items as ci join final.carts as c on c.ID = ci.Cart_ID
		WHERE ci.Resource_Type = 'Course' 
			and c.Purchased_At is not Null
	Group by 1
	having count(distinct ci.Resource_ID)>1
		)
select
	count(user_id)
from loyal_users`

ОТВЕТ 12656


Вы подготовили файл с данными по продажам курсов в разрезе пользователей (купивших более одного курса).

Код SQL для формирования базы


`With loyal_users as(
	Select
		c.User_ID
	from final.cart_items as ci join final.carts as c on c.ID = ci.Cart_ID
		WHERE ci.Resource_Type = 'Course' 
			and c.Purchased_At is not Null
	Group by 1
	having count(distinct ci.Resource_ID)>1
		)
select
	distinct lu.User_ID,
	ci.Resource_ID
from final.carts as c join loyal_users as lu
on c.User_ID=lu.User_ID
join final.cart_items as ci on c.ID = ci.Cart_ID
Where ci.Resource_Type = 'Course'
and c.Purchased_At is not Null`


In [216]:
# я перенес сформированную SQL-запросом базу на свой акканут GitHub на случай если учетные данные для подключения к базе SkilFactory изменятся
# Прочитаем таблицу и запишем в датафрем df

df = pd.read_csv('https://raw.githubusercontent.com/sgarifullin/FINAL_PROJECT_SkillFactory_Data_Analyst/main/clients_and_course.csv')


In [217]:
# Посмотрим на данные таблицы
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34074 entries, 0 to 34073
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   user_id      34074 non-null  int64
 1   resource_id  34074 non-null  int64
dtypes: int64(2)
memory usage: 532.5 KB


In [218]:
# Проверим вид таблицы
df.head()

,user_id,resource_id
0,909757,356
1,583850,515
2,1559882,566
3,970967,679
4,1640443,566


ПРЕОБРАЗОВАНИЕ ДАННЫХ

In [219]:
# Группируем df по user_id
user_resources = df.groupby('user_id')['resource_id'].apply(list)

# Создаем словарь: Ключ: user_id, значение: id курсов (resource_id)
user_resources_dict = user_resources.to_dict()
user_resources_dict

{51: [516, 1099],
 6117: [357, 1125, 356],
 10275: [553, 1147],
 10457: [1138, 361],
 17166: [356, 357],
 17186: [1140, 1125],
 22847: [551, 745],
 23205: [745, 553],
 23837: [551, 1138],
 23838: [553, 568],
 24640: [566, 517, 514],
 26191: [363, 511, 562, 563],
 30747: [745, 568],
 30789: [745, 509, 553],
 31157: [1125, 1144],
 31237: [509, 672, 568],
 32387: [552, 516],
 32508: [356, 552],
 32949: [357, 571],
 38783: [745, 568],
 42082: [509, 568, 516],
 44652: [513, 1141],
 47761: [1125, 571],
 53330: [552, 1138, 551, 862, 744],
 53594: [361, 1138],
 54190: [356, 679],
 54217: [1099, 571, 745],
 54956: [1099, 568, 509, 516],
 56388: [517, 750],
 58818: [800, 1125],
 59574: [569, 840],
 72101: [568, 509],
 76222: [745, 1125],
 76349: [514, 551, 509, 745],
 76488: [571, 765],
 76899: [745, 1187],
 76965: [566, 363],
 77811: [1125, 513],
 78450: [1103, 1100],
 81241: [502, 1103, 564, 865],
 84090: [566, 764],
 85070: [1139, 553, 568, 745],
 86123: [1144, 1125, 357],
 86955: [1186, 1144

In [220]:
# Создадим список из знайчений

grouped_courses = list(user_resources_dict.values())
grouped_courses # внутри списка содержаться списки покупок курсов на каждого пользователя

[[516, 1099],
 [357, 1125, 356],
 [553, 1147],
 [1138, 361],
 [356, 357],
 [1140, 1125],
 [551, 745],
 [745, 553],
 [551, 1138],
 [553, 568],
 [566, 517, 514],
 [363, 511, 562, 563],
 [745, 568],
 [745, 509, 553],
 [1125, 1144],
 [509, 672, 568],
 [552, 516],
 [356, 552],
 [357, 571],
 [745, 568],
 [509, 568, 516],
 [513, 1141],
 [1125, 571],
 [552, 1138, 551, 862, 744],
 [361, 1138],
 [356, 679],
 [1099, 571, 745],
 [1099, 568, 509, 516],
 [517, 750],
 [800, 1125],
 [569, 840],
 [568, 509],
 [745, 1125],
 [514, 551, 509, 745],
 [571, 765],
 [745, 1187],
 [566, 363],
 [1125, 513],
 [1103, 1100],
 [502, 1103, 564, 865],
 [566, 764],
 [1139, 553, 568, 745],
 [1144, 1125, 357],
 [1186, 1144, 1125],
 [366, 367, 513],
 [809, 519],
 [1100, 517],
 [1144, 515],
 [912, 1125, 356],
 [516, 489],
 [502, 356, 514, 489, 523],
 [765, 864, 862],
 [519, 523, 551, 679, 912, 672, 356],
 [564, 1100],
 [1125, 571, 912, 765],
 [523, 912, 519],
 [1146, 1187, 1147, 1139, 1099, 1101, 745],
 [516, 1125, 366, 51

In [221]:
# Создаем все возможнные комбинации пар курсов из подсписков grouped_courses
import itertools

grouped_courses
course_comb = []
for sublist in grouped_courses:
    sublist_combinations = list(itertools.combinations(sublist, 2))
    course_comb.extend(sublist_combinations)
    
course_comb

[(516, 1099),
 (357, 1125),
 (357, 356),
 (1125, 356),
 (553, 1147),
 (1138, 361),
 (356, 357),
 (1140, 1125),
 (551, 745),
 (745, 553),
 (551, 1138),
 (553, 568),
 (566, 517),
 (566, 514),
 (517, 514),
 (363, 511),
 (363, 562),
 (363, 563),
 (511, 562),
 (511, 563),
 (562, 563),
 (745, 568),
 (745, 509),
 (745, 553),
 (509, 553),
 (1125, 1144),
 (509, 672),
 (509, 568),
 (672, 568),
 (552, 516),
 (356, 552),
 (357, 571),
 (745, 568),
 (509, 568),
 (509, 516),
 (568, 516),
 (513, 1141),
 (1125, 571),
 (552, 1138),
 (552, 551),
 (552, 862),
 (552, 744),
 (1138, 551),
 (1138, 862),
 (1138, 744),
 (551, 862),
 (551, 744),
 (862, 744),
 (361, 1138),
 (356, 679),
 (1099, 571),
 (1099, 745),
 (571, 745),
 (1099, 568),
 (1099, 509),
 (1099, 516),
 (568, 509),
 (568, 516),
 (509, 516),
 (517, 750),
 (800, 1125),
 (569, 840),
 (568, 509),
 (745, 1125),
 (514, 551),
 (514, 509),
 (514, 745),
 (551, 509),
 (551, 745),
 (509, 745),
 (571, 765),
 (745, 1187),
 (566, 363),
 (1125, 513),
 (1103, 1100

In [222]:
from collections import Counter

unique_course_comb = [tuple(sorted(pair)) for pair in course_comb] # сортировка внутри пар
counter = Counter(unique_course_comb) # применение Counter
count = len(counter) # длина Counter и бужет число уникальных пар курсов 
display ('Число различных пар курсов: ', count) 


'Число различных пар курсов: '

3989

In [223]:
# Самая популярная пара курсов
display('Самая популярная пара курсов: ', counter.most_common(1)[0][0])

'Самая популярная пара курсов: '

(551, 566)

In [224]:
# Изменим объект счетчик на словарь

counter = counter.most_common()
counter_dict = dict(counter)
counter_dict

{(551, 566): 797,
 (515, 551): 417,
 (489, 551): 311,
 (523, 551): 304,
 (566, 794): 290,
 (489, 515): 286,
 (490, 566): 253,
 (490, 551): 247,
 (570, 752): 247,
 (569, 572): 216,
 (515, 523): 213,
 (553, 745): 212,
 (489, 523): 206,
 (569, 840): 204,
 (514, 551): 200,
 (516, 745): 199,
 (515, 566): 195,
 (489, 566): 188,
 (504, 572): 184,
 (572, 840): 178,
 (551, 552): 177,
 (507, 570): 172,
 (490, 809): 163,
 (489, 490): 152,
 (507, 752): 150,
 (523, 552): 144,
 (490, 515): 143,
 (551, 570): 142,
 (504, 569): 139,
 (514, 515): 139,
 (551, 745): 138,
 (514, 566): 138,
 (502, 551): 135,
 (504, 840): 135,
 (571, 1125): 122,
 (523, 566): 120,
 (502, 566): 120,
 (570, 809): 119,
 (752, 809): 115,
 (490, 523): 114,
 (357, 571): 112,
 (523, 564): 110,
 (551, 749): 109,
 (516, 553): 107,
 (551, 777): 107,
 (551, 679): 104,
 (356, 571): 103,
 (551, 564): 103,
 (515, 749): 103,
 (568, 745): 102,
 (356, 357): 100,
 (363, 511): 99,
 (551, 571): 98,
 (551, 809): 96,
 (502, 514): 95,
 (551, 794): 

In [225]:
# Проверим, что стип объекта десвительно - словарь.
type(counter_dict)

dict

In [226]:
# Создаем функцию, которая получает на входе номер курса, далее проверяет наличие курса в ключе словаря, 
# если есть сопадение - дабавляет id курса в список list_of_pair, далее этот список сортируется по значением словаря и выдает новый список sorted_courses
# на выходе функция возвращает первые две записи отсортированого списка, т.е. две первые пары с id курса с наиболее частыми покупками.
def find_pair(course_id):
    list_of_pair = []
    for key in counter_dict.keys():
        if course_id in key:
            list_of_pair.append((key, counter_dict[key]))
    sorted_courses  = sorted(list_of_pair, key=lambda x: x[1], reverse=True)
    return sorted_courses[:2]

In [227]:
# Создадим новый датафремй в котором будут все номера курсов и количество покупок этих курсов

course_recomend = pd.DataFrame({
    'course_id': df['resource_id'].unique(),
    'qty_of_buy': df['resource_id'].value_counts()
})
course_recomend = course_recomend.reset_index(drop=True)

# Создаим копию датафрейма с продажами курсов, понадобится позже для рекомендаций в низкачастотных парах
top_sales_course = course_recomend.copy()

top_sales_course.head()


top_sales_course_list = list(top_sales_course.course_id)
top_sales_course_list

[356,
 515,
 566,
 679,
 551,
 504,
 570,
 514,
 571,
 1129,
 503,
 523,
 568,
 1138,
 511,
 745,
 516,
 741,
 1181,
 569,
 764,
 572,
 794,
 490,
 489,
 564,
 502,
 753,
 750,
 829,
 791,
 752,
 776,
 563,
 908,
 1144,
 1128,
 357,
 1141,
 553,
 519,
 365,
 659,
 368,
 1125,
 840,
 863,
 358,
 809,
 1103,
 363,
 1184,
 756,
 360,
 507,
 1100,
 552,
 664,
 518,
 359,
 757,
 672,
 1156,
 765,
 1140,
 562,
 1188,
 509,
 749,
 366,
 1102,
 907,
 1152,
 1161,
 517,
 777,
 1139,
 670,
 1145,
 671,
 1101,
 361,
 1099,
 513,
 902,
 743,
 1115,
 1116,
 367,
 1187,
 912,
 814,
 1124,
 1186,
 742,
 909,
 1146,
 744,
 508,
 835,
 837,
 364,
 362,
 803,
 810,
 813,
 862,
 1160,
 800,
 1104,
 755,
 864,
 1147,
 834,
 1185,
 836,
 1198,
 1199,
 911,
 830,
 833,
 1182,
 865,
 866,
 1200,
 1201]

In [228]:
# Добавим в датафрем новый столбец с исвользованием функции course_recomend, которая будет на входе принимать course_id,
# а на выхоже получим столбец с наиболее частотными парами курсов с course_id
course_recomend['pair'] = course_recomend['course_id'].apply(find_pair)
course_recomend

,course_id,qty_of_buy,pair
0,356,2935,"[((356, 571), 103), ((356, 357), 100)]"
1,515,2342,"[((515, 551), 417), ((489, 515), 286)]"
2,566,1311,"[((551, 566), 797), ((566, 794), 290)]"
3,679,1125,"[((551, 679), 104), ((489, 679), 77)]"
4,551,1084,"[((551, 566), 797), ((515, 551), 417)]"
...,...,...,...
121,1182,3,"[((1182, 1187), 1), ((553, 1182), 1)]"
122,865,3,"[((829, 865), 7), ((865, 1103), 6)]"
123,866,2,"[((865, 866), 2), ((866, 1104), 2)]"
124,1200,2,"[((866, 1200), 1), ((1104, 1200), 1)]"


In [229]:
#Разобъем столбик pair на два отельных: 1st_recomend и 2nd_recomend
course_recomend['1st_recomend'] = course_recomend['pair'].apply(lambda x: x[0])
course_recomend['2nd_recomend'] = course_recomend['pair'].apply(lambda x: x[1])

# Уберем лишние стобцы
course_recomend = course_recomend.drop('pair', axis=1)
course_recomend = course_recomend.drop('qty_of_buy', axis=1)

course_recomend

,course_id,1st_recomend,2nd_recomend
0,356,"((356, 571), 103)","((356, 357), 100)"
1,515,"((515, 551), 417)","((489, 515), 286)"
2,566,"((551, 566), 797)","((566, 794), 290)"
3,679,"((551, 679), 104)","((489, 679), 77)"
4,551,"((551, 566), 797)","((515, 551), 417)"
...,...,...,...
121,1182,"((1182, 1187), 1)","((553, 1182), 1)"
122,865,"((829, 865), 7)","((865, 1103), 6)"
123,866,"((865, 866), 2)","((866, 1104), 2)"
124,1200,"((866, 1200), 1)","((1104, 1200), 1)"


In [230]:
# Разобъем столбцы рекомендаций на столбцы с парами и стобцы с частотой пар.
course_recomend['1st_recomend_count'] = course_recomend['1st_recomend'].apply(lambda x: x[1])
course_recomend['2nd_recomend_count'] = course_recomend['2nd_recomend'].apply(lambda x: x[1])
course_recomend['1st_recomend'] = course_recomend['1st_recomend'].apply(lambda x: x[0])
course_recomend['2nd_recomend'] = course_recomend['2nd_recomend'].apply(lambda x: x[0])
course_recomend

,course_id,1st_recomend,2nd_recomend,1st_recomend_count,2nd_recomend_count
0,356,"(356, 571)","(356, 357)",103,100
1,515,"(515, 551)","(489, 515)",417,286
2,566,"(551, 566)","(566, 794)",797,290
3,679,"(551, 679)","(489, 679)",104,77
4,551,"(551, 566)","(515, 551)",797,417
...,...,...,...,...,...
121,1182,"(1182, 1187)","(553, 1182)",1,1
122,865,"(829, 865)","(865, 1103)",7,6
123,866,"(865, 866)","(866, 1104)",2,2
124,1200,"(866, 1200)","(1104, 1200)",1,1


In [231]:
# Произведем двойную сортировку: с начала по столбцу 1st_recomend_count (частота первой рекомендации) - по убыванию,
# затем по 2nd_recomend_count - частота, второй рекомнедации.
course_recomend.sort_values(by=['1st_recomend_count', '2nd_recomend_count'], ascending=False, inplace=True)
course_recomend = course_recomend.reset_index(drop=True)

course_recomend

,course_id,1st_recomend,2nd_recomend,1st_recomend_count,2nd_recomend_count
0,551,"(551, 566)","(515, 551)",797,417
1,566,"(551, 566)","(566, 794)",797,290
2,515,"(515, 551)","(489, 515)",417,286
3,489,"(489, 551)","(489, 515)",311,286
4,523,"(523, 551)","(515, 523)",304,213
...,...,...,...,...,...
121,911,"(514, 911)","(776, 911)",2,1
122,902,"(777, 902)","(519, 902)",1,1
123,1182,"(1182, 1187)","(553, 1182)",1,1
124,1200,"(866, 1200)","(1104, 1200)",1,1


In [232]:
# Расчитываем два новых столбика с рекомендательным курсом
course_recomend['1st_recomendation'] = course_recomend.apply \
    (lambda row: row['1st_recomend'][0] if row['1st_recomend'][0] != row['course_id'] else row['1st_recomend'][1], axis=1)
course_recomend['2nd_recomendation'] = course_recomend.apply \
    (lambda row: row['2nd_recomend'][0] if row['2nd_recomend'][0] != row['course_id'] else row['2nd_recomend'][1], axis=1) 

# Удалляем столбики с парами
course_recomend = course_recomend.drop('1st_recomend', axis=1)
course_recomend = course_recomend.drop('2nd_recomend', axis=1)
    
course_recomend

,course_id,1st_recomend_count,2nd_recomend_count,1st_recomendation,2nd_recomendation
0,551,797,417,566,515
1,566,797,290,551,794
2,515,417,286,551,489
3,489,311,286,551,515
4,523,304,213,551,515
...,...,...,...,...,...
121,911,2,1,514,776
122,902,1,1,777,519
123,1182,1,1,1187,553
124,1200,1,1,866,1104


In [233]:
import plotly
import plotly.express as px

#line_data = covid_df.groupby('date', as_index=False).sum()
fig = px.line(
    data_frame=course_recomend, 
    y=['1st_recomend_count', '2nd_recomend_count'],
    height=400, 
    width=700, 
    title='График частотностей первой и второй рекомендации'
)
fig.show()

На графике наблюдаеистся весь массив частоностей первой и второй рекомендации.

In [234]:
display(course_recomend.describe())

,course_id,1st_recomend_count,2nd_recomend_count,1st_recomendation,2nd_recomendation
count,126.000000,126.000000,126.000000,126.000000,126.000000
mean,788.031746,79.436508,55.595238,649.190476,637.269841
std,270.113604,120.688458,71.667139,190.611467,199.320302
min,356.000000,1.000000,1.000000,361.000000,356.000000
25%,552.250000,16.250000,11.000000,551.000000,516.000000
50%,776.500000,37.500000,27.000000,566.000000,557.500000
75%,1101.750000,97.250000,67.250000,745.000000,748.750000
max,1201.000000,797.000000,417.000000,1187.000000,1144.000000


Данные по частоте очень сильно разбросаны вокург среднего значения 79 и 56 для первой и второй рекомендации, стандартное отклонение составляет 120 и 71 соответсвенно. 
Предлогаю рекомендации, где чатота меньше 10% перцентиля заменять на более популярные курсы.

In [235]:
# Создадим переменные с 90% перцентилем для 1й и 2й рекомендации  -в нее попадает частота встречаемая менее, чем в 90% случаев

import numpy as np

percentil_10_1st =  np.percentile(course_recomend['1st_recomend_count'], 10)
percentil_10_2nd =  np.percentile(course_recomend['2nd_recomend_count'], 10)

course_recomend

,course_id,1st_recomend_count,2nd_recomend_count,1st_recomendation,2nd_recomendation
0,551,797,417,566,515
1,566,797,290,551,794
2,515,417,286,551,489
3,489,311,286,551,515
4,523,304,213,551,515
...,...,...,...,...,...
121,911,2,1,514,776
122,902,1,1,777,519
123,1182,1,1,1187,553
124,1200,1,1,866,1104


В Датафреме встречаются комбинации курсов, продажи которых в комбинации слишком низкачастотны, целесообразно выдвать рекомендации по более популярным курсам, которые являются лидерами продаж - например,  ТОП-1 и ТОП-2 всех продаж курсов. 

In [236]:
# Функция, которая сравнивает значение частоты с персентилем, если оно меньше,
# то значение курса заменяется на ТОП-1 в перовй рекомендации и на ТОП-2 во второй рекомендации.

def replace_value_1(x):
    if x < percentil_10_1st:
        return top_sales_course_list[0]
    else:
        return x
    
def replace_value_2(x):
    if x < percentil_10_2nd:
        return top_sales_course_list[1]
    else:
        return x

In [237]:
# Применяем написаные выше функции к столбцам 1st_recomendation и 2nd_recomendation

course_recomend['1st_recomendation'] = course_recomend['1st_recomend_count'].apply(lambda x: replace_value_1(x))
course_recomend['2nd_recomendation'] = course_recomend['2nd_recomend_count'].apply(lambda x: replace_value_2(x))

# Удаляем лишние столбцы с частотой для формирования конечного датафрейма
course_recomend = course_recomend.drop('1st_recomend_count', axis=1)
course_recomend = course_recomend.drop('2nd_recomend_count', axis=1)


# Конечный датафрем - рекомндательная таблица
display (course_recomend)
# Запись датафрейма в course_recomend.xlsx файл
course_recomend.to_excel('course_recomend.xlsx', index=False)

,course_id,1st_recomendation,2nd_recomendation
0,551,797,417
1,566,797,290
2,515,417,286
3,489,311,286
4,523,304,213
...,...,...,...
121,911,356,515
122,902,356,515
123,1182,356,515
124,1200,356,515
